In [1]:
import requests
from datetime import datetime
import os
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import plotly.express as px

In [2]:
base_url = os.getenv("IMMICH_BASE_URL")
token = os.getenv("IMMICH_TOKEN")
year = os.getenv("YEAR")
start_date = f"{year}-01-01"
end_date = f"{year}-12-31"

headers = {"x-api-key": token}
response = requests.get(f"{base_url}/asset", headers=headers)

In [3]:
if response.status_code == 200:
    photos = response.json()
    print(len(photos))
    print(photos[200])
    # print(f"ID: {photos[0]['id']}, File Name: {photos[0]['fileName']}, Date: {photos[0]['uploadDate']}")
else:
    print(f"Error: {response.status_code} - {response.text}")


23291
{'id': '6325333d-4eaa-416d-b79b-a96f9fb7bbfa', 'deviceAssetId': '1000093396', 'ownerId': '50f9308f-c566-467c-a90d-0047c10a94c5', 'deviceId': '979ef84154fd407355810fbfbeb69870bc1a39edcf799a1248ff7f1b3542ba60', 'libraryId': '5d94339e-9fe6-44ae-a969-0b88f7606a47', 'type': 'IMAGE', 'originalPath': 'upload/upload/50f9308f-c566-467c-a90d-0047c10a94c5/be/e1/bee1f215-51e1-4f68-aabb-b590bb359be6.jpg', 'originalFileName': 'PXL_20250119_190741558.jpg', 'resized': True, 'thumbhash': 'HwgSDIAJWYl7dXZ4d4aIgHcKeQ==', 'fileCreatedAt': '2025-01-19T19:07:41.558Z', 'fileModifiedAt': '2025-01-19T19:07:44.000Z', 'localDateTime': '2025-01-19T11:07:41.558Z', 'updatedAt': '2025-01-20T20:27:18.312Z', 'isFavorite': False, 'isArchived': False, 'isTrashed': False, 'duration': '0:00:00.00000', 'exifInfo': {'make': 'Google', 'model': 'Pixel 7', 'exifImageWidth': 4032, 'exifImageHeight': 2268, 'fileSizeInByte': 1429018, 'orientation': '1', 'dateTimeOriginal': '2025-01-19T19:07:41.558Z', 'modifyDate': '2025-01-

In [4]:
photos_with_metadata = [photo for photo in photos if photo['exifInfo']['latitude'] != None]

In [5]:
print(f"{len(photos_with_metadata)}/{len(photos)}")

19553/23291


In [6]:
df = pd.DataFrame(photos_with_metadata)

In [7]:
exif_cols = pd.json_normalize(df.exifInfo)
df[exif_cols.columns] = exif_cols

In [8]:
df.columns

Index(['id', 'deviceAssetId', 'ownerId', 'deviceId', 'libraryId', 'type',
       'originalPath', 'originalFileName', 'resized', 'thumbhash',
       'fileCreatedAt', 'fileModifiedAt', 'localDateTime', 'updatedAt',
       'isFavorite', 'isArchived', 'isTrashed', 'duration', 'exifInfo',
       'livePhotoVideoId', 'tags', 'people', 'checksum', 'stackCount',
       'isOffline', 'isExternal', 'isReadOnly', 'hasMetadata', 'make', 'model',
       'exifImageWidth', 'exifImageHeight', 'fileSizeInByte', 'orientation',
       'dateTimeOriginal', 'modifyDate', 'timeZone', 'lensModel', 'fNumber',
       'focalLength', 'iso', 'exposureTime', 'latitude', 'longitude', 'city',
       'state', 'country', 'description', 'projectionType'],
      dtype='object')

In [9]:
filtered_df = df[(df['fileCreatedAt'] >= f"{year}-01-01") & (df['fileCreatedAt'] <= f"{year}-12-31")]

In [10]:
filtered_df = filtered_df[['fileCreatedAt', 'latitude', 'longitude', 'city', 'state', 'country']].iloc[::-1]

In [11]:
filtered_df['prev_latitude'] = [np.nan] + list(filtered_df['latitude'])[:-1]
filtered_df['prev_longitude'] = [np.nan] + list(filtered_df['longitude'])[:-1]

In [12]:
filtered_df

,fileCreatedAt,latitude,longitude,city,state,country,prev_latitude,prev_longitude
5310,2024-01-01T03:02:45.140Z,28.624675,-81.301600,Goldenrod,Florida,United States of America,NaN,NaN
5309,2024-01-01T05:48:20.039Z,28.638150,-81.302806,Goldenrod,Florida,United States of America,28.624675,-81.301600
5308,2024-01-01T05:48:31.339Z,28.638150,-81.302806,Goldenrod,Florida,United States of America,28.638150,-81.302806
5307,2024-01-01T06:02:07.000Z,28.636800,-81.303400,Goldenrod,Florida,United States of America,28.638150,-81.302806
5306,2024-01-01T06:08:09.000Z,28.637100,-81.303200,Goldenrod,Florida,United States of America,28.636800,-81.303400
...,...,...,...,...,...,...,...,...
475,2024-12-28T21:21:34.424Z,30.258975,-85.606722,Lynn Haven,Florida,United States of America,30.258975,-85.606722
474,2024-12-28T21:21:49.494Z,30.258975,-85.606722,Lynn Haven,Florida,United States of America,30.258975,-85.606722
473,2024-12-28T21:21:57.463Z,30.258975,-85.606722,Lynn Haven,Florida,United States of America,30.258975,-85.606722
472,2024-12-30T01:34:28.000Z,35.218000,-80.942200,Charlotte,North Carolina,United States of America,30.258975,-85.606722


In [13]:
filtered_df['distance_from_prev'] = filtered_df.apply(lambda point: geodesic((point['latitude'], point['longitude']), (point['prev_latitude'], point['prev_longitude'])).km if np.isfinite(point['prev_latitude']) else 0, axis=1)

In [14]:
final_df = filtered_df[filtered_df['distance_from_prev'] > 70]

In [15]:
final_df

,fileCreatedAt,latitude,longitude,city,state,country,prev_latitude,prev_longitude,distance_from_prev
5123,2024-01-21T00:24:49.969Z,28.593889,-81.384333,Fairview Shores,Florida,United States of America,29.182717,-81.038333,73.472353
5002,2024-01-27T23:05:46.614Z,28.093058,-82.781856,Palm Harbor,Florida,United States of America,29.193989,-81.073789,206.820706
4989,2024-01-29T15:39:14.006Z,29.188858,-81.048022,Daytona Beach,Florida,United States of America,28.078872,-82.765800,208.191788
4936,2024-02-10T02:01:20.010Z,28.456753,-81.494358,Doctor Phillips,Florida,United States of America,29.180611,-81.038419,91.743763
4918,2024-02-12T15:03:16.565Z,29.188125,-81.050389,Daytona Beach,Florida,United States of America,28.538569,-81.378217,78.777849
4898,2024-02-20T00:45:42.617Z,28.589297,-81.364672,Winter Park,Florida,United States of America,29.182719,-81.038369,73.068162
4896,2024-02-20T06:37:32.506Z,29.182719,-81.038369,Daytona Beach,Florida,United States of America,28.589294,-81.364806,73.074106
4809,2024-03-09T18:57:47.997Z,28.591161,-81.204025,Alafaya,Florida,United States of America,29.221700,-81.097000,70.660449
4785,2024-03-10T16:36:41.440Z,28.519094,-80.681611,Port Saint John,Florida,United States of America,29.223919,-81.101700,88.216101
4760,2024-03-11T23:45:51.854Z,29.188856,-81.048047,Daytona Beach,Florida,United States of America,28.408150,-80.594956,97.178222


In [16]:
unique_cities = final_df.groupby(['city', 'state']).ngroups
unique_states = final_df['state'].nunique()
unique_countries = final_df['country'].nunique()
total_distance_traveled = round(final_df['distance_from_prev'].sum(), 2)
most_common_state = final_df['state'].mode().item()

In [17]:
print(f"{year} Stats")
print(f"{unique_cities} unique cities")
print(f"{unique_states} unique states")
print(f"{unique_countries} unique countries")
print(f"{total_distance_traveled}km total distance traveled")
print(f"{most_common_state} is the most visited state")

2024 Stats
26 unique cities
8 unique states
1 unique countries
21413.0km total distance traveled
Florida is the most visited state


In [105]:
i = 0
group = 0
animated_json = {'latitude': [], 'longitude': [], 'fileCreatedAt': [], 'city': [], 'state': [], 'country': [], 'group': []}
for index, row in final_df.iterrows():
    animated_json['latitude'].append(row['latitude'])
    animated_json['longitude'].append(row['longitude'])
    animated_json['fileCreatedAt'].append(row['fileCreatedAt'])
    animated_json['city'].append(row['city'])
    animated_json['state'].append(row['state'])
    animated_json['country'].append(row['country'])
    animated_json['group'].append(group)

    if i > 0 and i < len(final_df) - 1:
        group+=1
        animated_json['latitude'].append(row['latitude'])
        animated_json['longitude'].append(row['longitude'])
        animated_json['fileCreatedAt'].append(row['fileCreatedAt'])
        animated_json['city'].append(row['city'])
        animated_json['state'].append(row['state'])
        animated_json['country'].append(row['country'])
        animated_json['group'].append(group)

    i+=1
    


In [107]:
animated_df = pd.DataFrame(animated_json)

In [122]:
fig = px.line_map(animated_df,
            lat='latitude',
            lon='longitude',
            animation_frame='group',
            zoom=4,
            width=1200,
            height=800)

In [126]:
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 2000
fig.show()

In [38]:
fig.write_html("test.html", auto_play=False)

In [63]:
import pandas as pd
import plotly.express as px
import numpy as np

data = [
    ["Year", "Country", "Output", "Investment", "Depreciation"],
    ["2020", "Netherlands", 1, 2, 1],
    ["2021", "Netherlands", 2, 2, 1],
    ["2022", "Netherlands", 3, 2, 1],
    ["2023", "Netherlands", 4, 2, 1],
]
df = pd.DataFrame(data[1:], columns=data[0])
df = pd.DataFrame(
    {
        c: list(range(2000 if i == 0 else 1, 2050 if i == 0 else 51, 1))
        if c in ["Year", "Output"]
        else np.full(50, i if c != "Country" else "Netherlands")
        for i, c in enumerate(data[0])
    }
).assign(Decade=lambda d: d["Year"] // 10 - 200)

px.line(
    df,
    x="Year",
    y="Output",
    color="Country",
    title="Macroeconomic variables over time",
    range_x=[df["Year"].iloc[0], df["Year"].iloc[-1]],
    range_y=[0, max(df["Output"]) * 1.25],
    animation_frame="Decade",
)

In [64]:
df

,Year,Country,Output,Investment,Depreciation,Decade
0,2000,Netherlands,1,3,4,0
1,2001,Netherlands,2,3,4,0
2,2002,Netherlands,3,3,4,0
3,2003,Netherlands,4,3,4,0
4,2004,Netherlands,5,3,4,0
5,2005,Netherlands,6,3,4,0
6,2006,Netherlands,7,3,4,0
7,2007,Netherlands,8,3,4,0
8,2008,Netherlands,9,3,4,0
9,2009,Netherlands,10,3,4,0
